In [79]:
import urllib.request as url
import ssl
from multiprocessing import Pool
from bs4 import BeautifulSoup as BS
import time
import os
import re

# Obter atas do Copom
### Repositório do Github: https://github.com/paulodt/TCC-PauloDeTarso

## Código para transformar os pdfs em texto

Criado a partir da modificação de:

http://stackoverflow.com/questions/5725278/how-do-i-use-pdfminer-as-a-library

In [84]:
from pdfminer.pdfinterp import PDFResourceManager, process_pdf
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO

def convert_pdf(path):

    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    process_pdf(rsrcmgr, device, fp)
    fp.close()
    device.close()

    str = retstr.getvalue()
    retstr.close()
    return str

def clean_string(s):
    #Remover símbolos
    s = re.sub(r'[^\w]', ' ', s)
    #Trocar espaço múltiplo por simples
    s = re.sub( '\s+', ' ', s)
    return s

## Baixar as atas do COPOM

Para ignorar a verificação de certificados, me baseio em:
http://stackoverflow.com/questions/36600583/python-3-urllib-ignore-ssl-certificate-verification

In [3]:
#Ignorar a verificação de certificados no "urlopen"
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

Aqui, crio uma função para tratar de cada ata e poder paralelizar o processo.

Se a ata estiver em pdf, utilizo o programa "convert_pdf" acima para transformar em um txt. Se for um html, transformo em um txt diretamente.

In [28]:
#Parâmetros
total_atas = 207
atas = range(total_atas)
path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

for n_ata in atas:
    try:
        #Progresso
        print("Progresso: " + str(round(100*n_ata/total_atas)) + "%",end="\r")
        
        #Parâmetros
        ata = str(n_ata)
        link = "https://www.bcb.gov.br/?COPOM" + ata
        filename = "COPOM_" + ata

        #Acessar o site e obter o tipo (html ou PDF)
        site = url.urlopen(link,context=ctx)
        content_type = dict(site.getheaders())["Content-Type"]

        #Checar se é um PDF ou um HTML
        if "pdf" in content_type:

            #Baixar o arquivo
            with open(path + filename + ".pdf","wb") as file_in:
                file_in.write(site.read())
            file_in.close()

            full_text = convert_pdf(filename + ".pdf")

            #Escrever no arquivo txt
            file_out = open(path + filename + ".txt","w")
            file_out.write(full_text[full_text.index("Horário de início"):])
            file_out.close()

            #Remover o arquivo original
            os.remove(path + filename + ".pdf")

        else:
            soup = BS(site.read(),"lxml")
            full_text = "\n".join(soup.findAll(text=True))

            #Escrever no arquivo txt
            file_out = open(path + filename + ".txt","w")
            
            try:
                full_text = full_text[full_text.index("Data:"):]
            except:
                full_text = full_text[full_text.index("Data\n:"):]
                
            file_out.write(full_text)
            file_out.close()
    
    except:
        continue

In [53]:
#Parâmetros
inicio_atas = 21
final_atas = 206
total_atas = final_atas - inicio_atas + 1
atas = range(inicio_atas,total_atas)

path = "D:/Users/paulotarsosantos/Documents/Documentos/TCC/Atas/"

for n_ata in atas:
    #Progresso
    print("Progresso: " + str(round(100*(n_ata-inicio_atas)/total_atas)) + "%",end="\r")

    #Parâmetros
    ata = str(n_ata)
    link = "https://www.bcb.gov.br/?COPOM" + ata
    filename = "COPOM_" + ata

    #Acessar o site e obter o tipo (html ou PDF)
    site = url.urlopen(link,context=ctx)
    full_text = site.read()

    file_out = open(path + filename + ".txt","wb")
    try:
        print(ata)
        file_out.write(full_text[full_text.find(bytes("Data:",'utf-8')):])
    except:
        print(ata)
        file_out.write(full_text[full_text.find(bytes("Data\n:",'utf-8')):])
    file_out.close()

21ogresso: 0%
22ogresso: 1%
23ogresso: 1%
24ogresso: 2%
25ogresso: 2%
26ogresso: 3%
27ogresso: 3%
28ogresso: 4%
29ogresso: 4%
30ogresso: 5%
31ogresso: 5%
32ogresso: 6%
33ogresso: 6%
34ogresso: 7%
35ogresso: 8%
36ogresso: 8%
37ogresso: 9%
38ogresso: 9%
39ogresso: 10%
40ogresso: 10%
41ogresso: 11%
42ogresso: 11%
43ogresso: 12%
44ogresso: 12%
45ogresso: 13%
46ogresso: 13%
47ogresso: 14%
48ogresso: 15%
49ogresso: 15%
50ogresso: 16%
51ogresso: 16%
52ogresso: 17%
53ogresso: 17%
54ogresso: 18%
55ogresso: 18%
56ogresso: 19%
57ogresso: 19%
58ogresso: 20%
59ogresso: 20%
60ogresso: 21%
61ogresso: 22%
62ogresso: 22%
63ogresso: 23%
64ogresso: 23%
65ogresso: 24%
66ogresso: 24%
67ogresso: 25%
68ogresso: 25%
69ogresso: 26%
70ogresso: 26%
71ogresso: 27%
72ogresso: 27%
73ogresso: 28%
74ogresso: 28%
75ogresso: 29%
76ogresso: 30%
77ogresso: 30%
78ogresso: 31%
79ogresso: 31%
80ogresso: 32%
81ogresso: 32%
82ogresso: 33%
83ogresso: 33%
84ogresso: 34%
85ogresso: 34%
86ogresso: 35%
87ogresso: 35%
88ogresso: 36

In [55]:
for ata in range(21,207):
    file = open("Atas/COPOM_" + str(ata) + ".txt")
    if file.read() == "":
        print("Arquivo da reunião de número " + str(ata) + " está vazio!")

Arquivo da reunião de número 186 está vazio!
Arquivo da reunião de número 187 está vazio!
Arquivo da reunião de número 188 está vazio!
Arquivo da reunião de número 189 está vazio!
Arquivo da reunião de número 190 está vazio!
Arquivo da reunião de número 191 está vazio!
Arquivo da reunião de número 192 está vazio!
Arquivo da reunião de número 193 está vazio!
Arquivo da reunião de número 194 está vazio!
Arquivo da reunião de número 195 está vazio!
Arquivo da reunião de número 196 está vazio!
Arquivo da reunião de número 197 está vazio!
Arquivo da reunião de número 198 está vazio!
Arquivo da reunião de número 199 está vazio!
Arquivo da reunião de número 200 está vazio!
Arquivo da reunião de número 201 está vazio!
Arquivo da reunião de número 202 está vazio!
Arquivo da reunião de número 203 está vazio!
Arquivo da reunião de número 204 está vazio!
Arquivo da reunião de número 205 está vazio!
Arquivo da reunião de número 206 está vazio!


In [77]:
ata = str(112)
link = "https://www.bcb.gov.br/?COPOM" + ata
filename = "COPOM_" + ata
site = url.urlopen(link,context=ctx)
soup = BS(site,"lxml",from_encoding='utf-8')
full_text = "\n".join(soup.findAll(text=True))

full_text = full_text[full_text.find(bytes("Data","ascii")):]
full_text

TypeError: must be str, not bytes

In [85]:
clean_string(full_text)

' Antes da Trilha 19 32 37 Depois da Trilha 19 32 37 HTML PUBLIC W3C DTD HTML 4 01 Transitional EN http www w3 org TR html4 loose dtd document ready function finiciar trackGA document title 112ª Reunião var t_idioma P Zona se Salto DosVOX Página principal do site Notícias Mapa do site Perguntas Frequentes Ajuda Ouvidoria Política de privacidade Fale conosco Detalhes das teclas de atalho Saltar para o início da página corrente Saltar para o menu principal Saltar para o novo conteúdo Portal do Governo Brasileiro Atualize sua Barra de Governo Sobre a Instituição Perguntas frequentes Glossário Mapa do site Sisbacen Fale conosco Links English function utf8_encode argString http kevin vanzonneveld net original by Webtoolkit info http www webtoolkit info improved by Kevin van Zonneveld http kevin vanzonneveld net improved by sowberry tweaked by Jack bugfixed by Onno Marsman improved by Yves Sucaet bugfixed by Onno Marsman bugfixed by Ulrich bugfixed by Rafal Kukawski improved by kirilloid exa